<a href="https://colab.research.google.com/github/pavangelika/Student-s-daily/blob/main/Student's_daily.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Установка библиотеки plotly. Запсукаем код ниже

In [120]:
pip install plotly

Для расчета стипендии введите максимальную сумму, которую ребенок получит, если закончит четверть без троек.

In [121]:
money = 8000

In [122]:
import pandas as pd
import re
import plotly.graph_objects as go
import numpy as np

global_data={}

def process_grade(grade):
    """
    Обрабатывает оценки в виде строк, например '5/5', '22', '2 2' и т.п.
    Преобразует их в соответствующие числа или кортежи.
    """
    if isinstance(grade, str):
        # Проверка на оценку типа '22', '2 2' и т.п.
        parts = re.findall(r'\d+', grade)
        if len(parts) > 1 and all(2 <= int(x) <= 5 for x in parts):
            return int(parts[0]), int(parts[1])

        # Проверка на оценку типа '5/5' или '3/4'
        if '/' in grade:
            parts = grade.split('/')
            if len(parts) == 2 and all(part.isdigit() for part in parts):
                return int(parts[0]), int(parts[1])

        # Проверка на оценку типа '2 ОП'
        match = re.search(r'\d+', grade)
        if match:
            return int(match.group(0))

    if isinstance(grade, int):
        if grade > 5:
            # Разделяем число на цифры и возвращаем кортеж
            digits = tuple(int(d) for d in str(grade))
            return digits

    # Если это уже не строка или не подходит для обработки, возвращаем значение как есть
    return grade


def read_excel_and_process(file_path):
    """
    Читает данные из Excel-файла, очищает их, обрабатывает и возвращает результат.
    """
    try:
        # Чтение данных из Excel-файла
        df = pd.read_excel(file_path)
        df = df.dropna(how='all')  # Удаляем строки, где все значения пустые

        # Очистка данных
        df = df.apply(lambda x: x.map(lambda y: re.sub(r'\s*Н\s*|\xa0|\n', '', str(y)).strip() if isinstance(y, str) else y))
        df.replace(r"\s*Н\s*", "", regex=True, inplace=True)

        # Преобразуем DataFrame в словарь { "Предмет": [оценки] }
        grades_dict = df.set_index(df.columns[0]).apply(lambda x: x.dropna().tolist(), axis=1).to_dict()

        # Обработка оценок
        for subject, marks in grades_dict.items():
            processed_marks = []
            for mark in marks:
                processed_mark = process_grade(mark)  # Обрабатываем оценку
                if isinstance(processed_mark, tuple):  # Если оценка возвращает кортеж
                    processed_marks.extend(processed_mark)  # Распаковываем кортеж
                else:
                    processed_marks.append(processed_mark)  # Добавляем одиночную оценку
                # print(f"{processed_mark} {type(processed_mark)}")
            grades_dict[subject] = processed_marks

        # Поиск информации о четверти
        match = re.search(r'·\s*(I{1,3}\s+четверть)\s*', file_path)
        chart = match.group(1) if match else None

        # Формирование итогового результата
        result = []
        for subject, marks in grades_dict.items():
            unpacked_marks = []
            for mark in marks:
                # print(f"{mark} - {type(mark)}")

                if isinstance(mark, tuple):  # Проверка на кортеж
                    unpacked_marks.extend(mark)  # Распаковка значений кортежа
                else:
                    unpacked_marks.append(mark)  # Добавление обычной оценки
            filtered_data = [item for item in unpacked_marks if item != '' and item != '']
            result.append(f'{subject}: {", ".join(map(str, filtered_data))}')

        return result, chart

    except FileNotFoundError:
        print(f"Файл не найден: {file_path}")
        return [], None
    except Exception as e:
        print(f"Произошла ошибка: {e}")
        return [], None

def create_table(file_path):
    """
    Создает и отображает таблицу с оценками, а также сохраняет данные в глобальный словарь.
    """
    global global_data  # Используем глобальный словарь

    # Получаем данные из файла
    result, quarter_name = read_excel_and_process(file_path)
    quarter_name_short = quarter_name.replace("четверть", "ч").strip()

    # Преобразуем данные в словарь
    data = {}
    for line in result:
        subject, grades = line.split(": ")
        data[subject] = [int(grade) for grade in grades.split(", ") if grade.strip()]

    # Создаем DataFrame
    df = pd.DataFrame(list(data.items()), columns=["Предмет", "Оценки"])

    # Добавляем нумерацию с 1
    df.index = df.index + 1

    # Вычисляем средний балл с округлением до двух знаков
    df[f"{quarter_name_short}."] = df["Оценки"].apply(lambda x: round(sum(x) / len(x), 2))

    # Функция для корректного округления до целого
    def round_grade(avg):
        if avg >= 4.5:
            return 5
        elif avg >= 3.5:
            return 4
        elif avg >= 2.5:
            return 3
        elif avg >= 1.5:
            return 2
        else:
            return 1

    # Применяем функцию округления
    df[quarter_name_short] = df[f"{quarter_name_short}."].apply(round_grade)

    # Функция для определения коэффициента предмета
    def calculate_coefficient(subject):
        subject = str(subject).lower()
        if any(x in subject for x in ['алгебра', 'русский язык', 'литература']):
            return 1.3
        elif any(x in subject for x in ['изо', 'музыка', 'родной язык', 'русский (родной) язык',
                                      'родной (русский) язык', 'труд', 'технология', 'физкультура',
                                      'физическая культура']):
            return 0.1
        else:
            return 1.0

    # Добавляем столбец с коэффициентами (не будет отображаться)
    df['Коэф'] = df['Предмет'].apply(calculate_coefficient)

    # Рассчитываем общую сумму коэффициентов
    total_coefficient_sum = df['Коэф'].sum()

    # Рассчитываем базовую сумму без округления
    base_amounts = money * df['Коэф'] / total_coefficient_sum

    # Округляем до целых с коррекцией суммы
    rounded_amounts = base_amounts.round().astype(int)

    # Корректируем последнее значение для точной суммы money
    rounded_amounts.iloc[-1] += money - rounded_amounts.sum()

    df['руб.план'] = rounded_amounts  # Не будет отображаться

    # Рассчитываем руб.факт (руб.план если оценка >=3, иначе -руб.план)
    df['руб.факт'] = df.apply(lambda row: row['руб.план'] if row[quarter_name_short] > 3 else -row['руб.план'], axis=1)

    # Добавляем строку "ИТОГО"
    total_row = pd.DataFrame({
        "Предмет": ["ИТОГО"],
        "Оценки": [""],
        f"{quarter_name_short}.": [""],
        quarter_name_short: [""],
        "руб.факт": [df['руб.факт'].sum()]
    })

    # Выбираем только нужные столбцы для отображения
    display_columns = ["Предмет", "Оценки", f"{quarter_name_short}.", quarter_name_short, "руб.факт"]
    display_df = pd.concat([df[display_columns], total_row], ignore_index=True)

    # Форматируем оценки для вывода (без квадратных скобок)
    display_df["Оценки"] = display_df["Оценки"].apply(lambda x: ", ".join(map(str, x)) if isinstance(x, list) else x)

    # Отображаем таблицу только с нужными столбцами
    display(display_df)

# Сохраняем данные в глобальный словарь в исходной структуре
    global_data[quarter_name_short] = []
    for index, row in df.iterrows():
        global_data[quarter_name_short].append({
            "предмет": row["Предмет"],
            "средняя оценка": row[f"{quarter_name_short}."],
            "оценка": int(row[quarter_name_short]),
            "коэффициент": float(row['Коэф']),  # Добавляем новый параметр
            "руб_план": int(row['руб.план']),  # Добавляем новый параметр
            "руб_факт": int(row['руб.факт'])   # Добавляем новый параметр
        })

    # Сохраняем итоговую стипендию отдельно
    global_data[f"{quarter_name_short}_стипендия"] = int(df['руб.факт'].sum())

Выгружаем оценки из Электронного журнала за четверть на свой гугл диск в папку, например: Оценки/7 класс по четвертям/. Меняем в коде ниже значение student_name на ФИО вашего ребенка ваши student_name = "Иванов Иван Иванович" проверяем путь данные base_path = "/content/drive/MyDrive/Оценки/7 класс по четвертям/Отметки из журнала_ {} · {} четверть.xlsx"

In [123]:
from google.colab import userdata

# student_name заменить на ФИО ребенка например "Иванов Иван Иванович"
student_name = f"{userdata.get('surname')} {userdata.get('name')} {userdata.get('lastname')}"

# Базовый путь
base_path = "/content/drive/MyDrive/Оценки/7 класс по четвертям/Отметки из журнала_ {} · {} четверть.xlsx".format(student_name, "{}")


# Список четвертей
quarters = ["I", "II", "III", "IV"]

# Инициализация глобального словаря
global_data = {}

# Обработка каждого файла
for quarter in quarters:
    file_path = base_path.format(quarter)
    print('================================================================================')
    print('================================================================================')

    try:
        create_table(file_path)
    except FileNotFoundError:
        print(f"Файл не найден: {file_path}")
    except Exception as e:
        print(f"Произошла ошибка при обработке файла {file_path}: {e}")


,Предмет,Оценки,I ч.,I ч,руб.факт
0,Алгебра,"3, 3, 4, 2, 3, 2, 3, 3, 5, 3",3.1,3,-776
1,Англ. яз.,"4, 4, 2, 4, 2, 4, 5, 5",3.75,4,597
2,Биология,"4, 5, 3",4.0,4,597
3,Вероятность и статистика,"3, 3, 4, 4",3.5,4,597
4,География,"4, 3, 4, 4, 2, 4",3.5,4,597
5,Геометрия,"4, 5, 3",4.0,4,597
6,ИЗО,"5, 5, 5, 5",5.0,5,60
7,Информатика,"5, 4, 5",4.67,5,597
8,История,"4, 4, 4, 5, 2",3.8,4,597
9,Литература,"5, 2, 4, 2, 4",3.4,3,-776


,Предмет,Оценки,II ч.,II ч,руб.факт
0,Алгебра,"4, 3, 4, 3, 2, 3, 3, 3",3.12,3,-776
1,Англ. яз.,"3, 5, 5, 4",4.25,4,597
2,Биология,"5, 3, 3, 3, 3, 4, 4",3.57,4,597
3,Вероятность и статистика,"2, 5, 4, 4",3.75,4,597
4,География,"4, 3, 3, 3, 3, 4",3.33,3,-597
5,Геометрия,"2, 5, 4, 4, 3",3.6,4,597
6,ИЗО,"5, 5, 3, 4, 4, 5",4.33,4,60
7,Информатика,"4, 5, 5, 2, 4, 3",3.83,4,597
8,История,"4, 5, 2, 5",4.0,4,597
9,Литература,"5, 2, 4",3.67,4,776


,Предмет,Оценки,III ч.,III ч,руб.факт
0,Алгебра,"3, 4, 4, 4, 4",3.8,4,776
1,Англ. яз.,"4, 3, 3, 3, 5, 3, 4, 4, 3, 3",3.5,4,597
2,Биология,"2, 4, 5, 4",3.75,4,597
3,Вероятность и статистика,"4, 5, 4, 4",4.25,4,597
4,География,"4, 4, 4",4.0,4,597
5,Геометрия,"4, 4, 4, 5",4.25,4,597
6,ИЗО,"5, 5, 5",5.0,5,60
7,Информатика,"4, 4, 4",4.0,4,597
8,История,"4, 4, 4",4.0,4,597
9,Литература,"3, 4, 4",3.67,4,776


Файл не найден: /content/drive/MyDrive/Оценки/7 класс по четвертям/Отметки из журнала_ Павлюкова Полина Денисовна · IV четверть.xlsx
Произошла ошибка при обработке файла /content/drive/MyDrive/Оценки/7 класс по четвертям/Отметки из журнала_ Павлюкова Полина Денисовна · IV четверть.xlsx: 'NoneType' object has no attribute 'replace'


In [101]:
print(global_data)

{'I ч': [{'предмет': 'Алгебра', 'средняя оценка': 3.1, 'оценка': 3, 'коэффициент': 1.3, 'руб_план': 776, 'руб_факт': -776}, {'предмет': 'Англ. яз.', 'средняя оценка': 3.75, 'оценка': 4, 'коэффициент': 1.0, 'руб_план': 597, 'руб_факт': 597}, {'предмет': 'Биология', 'средняя оценка': 4.0, 'оценка': 4, 'коэффициент': 1.0, 'руб_план': 597, 'руб_факт': 597}, {'предмет': 'Вероятность и статистика', 'средняя оценка': 3.5, 'оценка': 4, 'коэффициент': 1.0, 'руб_план': 597, 'руб_факт': 597}, {'предмет': 'География', 'средняя оценка': 3.5, 'оценка': 4, 'коэффициент': 1.0, 'руб_план': 597, 'руб_факт': 597}, {'предмет': 'Геометрия', 'средняя оценка': 4.0, 'оценка': 4, 'коэффициент': 1.0, 'руб_план': 597, 'руб_факт': 597}, {'предмет': 'ИЗО', 'средняя оценка': 5.0, 'оценка': 5, 'коэффициент': 0.1, 'руб_план': 60, 'руб_факт': 60}, {'предмет': 'Информатика', 'средняя оценка': 4.67, 'оценка': 5, 'коэффициент': 1.0, 'руб_план': 597, 'руб_факт': 597}, {'предмет': 'История', 'средняя оценка': 3.8, 'оценка'

In [102]:
def show_global_data_arifmetic():
    """
    Выводит глобальный словарь в виде таблицы с учетом требований.
    """
    global global_data

    # Преобразуем глобальный словарь в DataFrame
    data_for_table = []
    for quarter, data in global_data.items():
        if isinstance(data, list):  # Проверяем, что это список предметов
            for subject_data in data:
                data_for_table.append({
                    "Предмет": subject_data["предмет"],
                    f'{quarter.replace("четверть", "ч").strip()}.': subject_data["средняя оценка"],
                })

    # Создаем DataFrame
    df = pd.DataFrame(data_for_table)

    # Группируем данные по предметам
    df = df.groupby("Предмет").first().reset_index()

    # Добавляем нумерацию с 1
    df.index = df.index + 1

    # Формируем одноуровневые заголовки
    columns = ["Предмет"]
    quarters = ['I ч', 'II ч', 'III ч', 'IV ч']

    for quarter in quarters:
        if any(q in global_data for q in [quarter, quarter.replace(" ч", " четверть")]):
            columns.append(f'{quarter}')
        else:
            columns.append(f'{quarter}')
            df[quarter] = np.nan

    columns.append("Годовая")
    df["Годовая"] = np.nan
    df.columns = columns

    # Рассчитываем годовую оценку
    for index, row in df.iterrows():
        scores = [row[col] for col in columns[1:-1] if not pd.isna(row[col])]
        if len(scores) >= 3:
            df.at[index, 'Годовая'] = int(round(np.mean(scores)))

    df['Годовая'] = df['Годовая'].astype(pd.Int64Dtype())

    title = f'{full_name}: среднее значение оценок за {class_name}. Прогноз годовой оценки'
    print('='*100)
    print(title)
    print('='*100)
    display(df)

def show_global_data_int():
    """
    Выводит глобальный словарь в виде таблицы с целыми оценками
    """
    global global_data

    data_for_table = []
    for quarter, data in global_data.items():
        if isinstance(data, list):  # Проверяем, что это список предметов
            for subject_data in data:
                data_for_table.append({
                    "Предмет": subject_data["предмет"],
                    f'{quarter.replace("четверть", "ч").strip()}': subject_data["оценка"],
                })

    df = pd.DataFrame(data_for_table)
    df = df.groupby("Предмет").first().reset_index()
    df.index = df.index + 1

    columns = ["Предмет"] + ['I ч', 'II ч', 'III ч', 'IV ч', 'Годовая']
    for col in ['I ч', 'II ч', 'III ч', 'IV ч']:
        if col not in df.columns:
            df[col] = np.nan

    df['Годовая'] = np.nan
    for index, row in df.iterrows():
        scores = [row[col] for col in ['I ч', 'II ч', 'III ч', 'IV ч'] if not pd.isna(row[col])]
        if len(scores) >= 3:
            df.at[index, 'Годовая'] = int(round(np.mean(scores)))

    for col in ['I ч', 'II ч', 'III ч', 'IV ч', 'Годовая']:
        df[col] = df[col].astype(pd.Int64Dtype())

    title = f'{full_name}: оценки за {class_name}. Прогноз годовой оценки'
    print('='*100)
    print(title)
    print('='*100)
    display(df)

# Выводим таблицы
show_global_data_arifmetic()
show_global_data_int()

Павлюкова Полина Денисовна: среднее значение оценок за 7 класс по четвертям. Прогноз годовой оценки


,Предмет,I ч,II ч,III ч,IV ч,Годовая
1,Алгебра,3.10,3.12,3.80,NaN,3
2,Англ. яз.,3.75,4.25,3.50,NaN,4
3,Биология,4.00,3.57,3.75,NaN,4
4,Вероятность и статистика,3.50,3.75,4.25,NaN,4
5,География,3.50,3.33,4.00,NaN,4
6,Геометрия,4.00,3.60,4.25,NaN,4
7,ИЗО,5.00,4.33,5.00,NaN,5
8,Информатика,4.67,3.83,4.00,NaN,4
9,История,3.80,4.00,4.00,NaN,4
10,Литература,3.40,3.67,3.67,NaN,4


Павлюкова Полина Денисовна: оценки за 7 класс по четвертям. Прогноз годовой оценки


,Предмет,I ч,II ч,III ч,IV ч,Годовая
1,Алгебра,3,3,4,<NA>,3
2,Англ. яз.,4,4,4,<NA>,4
3,Биология,4,4,4,<NA>,4
4,Вероятность и статистика,4,4,4,<NA>,4
5,География,4,3,4,<NA>,4
6,Геометрия,4,4,4,<NA>,4
7,ИЗО,5,4,5,<NA>,5
8,Информатика,5,4,4,<NA>,4
9,История,4,4,4,<NA>,4
10,Литература,3,4,4,<NA>,4


In [106]:
import plotly.graph_objects as go

# Отфильтровываем только ключи с оценками (исключаем ключи со стипендиями)
grade_quarters = [q for q in global_data.keys() if not q.endswith('_стипендия')]

title_avg = f'{full_name}: средние оценки за {class_name}'
title_final = f'{full_name}: оценки за {class_name} с округлением до целого числа'

# Берем предметы из первой четверти (они одинаковые для всех четвертей)
subjects = [item['предмет'] for item in global_data[grade_quarters[0]]]

# Данные для графиков
data_avg = {q: [item['средняя оценка'] for item in global_data[q]] for q in grade_quarters}
data_final = {q: [item['оценка'] for item in global_data[q]] for q in grade_quarters}

# Спокойные и приглушенные цвета в одной тональности (оттенки синего)
colors = ['#A1B6D7', '#7F98B2', '#5E7A8D', '#4A6472']  # Светло-синий, средний синий, темный синий, темно-серый синий

# 1. Гистограмма средних оценок
fig_avg = go.Figure()
for i, q in enumerate(grade_quarters):
    fig_avg.add_trace(go.Bar(
        x=subjects,
        y=data_avg[q],
        name=q,
        marker_color=colors[i % len(colors)]  # Цвета идут по кругу
    ))

fig_avg.update_layout(
    title=title_avg,
    xaxis_title='Предмет',
    yaxis_title='Средняя оценка',
    yaxis=dict(range=[2.9, 5]),
    barmode='group',
    shapes=[
        # Линия тренда на отметке 4 (темно-серый цвет)
        {
            'type': 'line',
            'x0': -0.5,  # Начало линии по оси X
            'x1': len(subjects) - 0.5,  # Конец линии по оси X
            'y0': 4,
            'y1': 4,
            'line': {
                'color': '#707070',  # Темно-серый цвет
                'width': 2,
                'dash': 'dashdot'  # Пунктирная линия
            }
        }
    ]
)

# 2. Гистограмма четвертных оценок
fig_final = go.Figure()
for i, q in enumerate(grade_quarters):
    fig_final.add_trace(go.Bar(
        x=subjects,
        y=data_final[q],
        name=q,
        marker_color=colors[i % len(colors)]
    ))

fig_final.update_layout(
    title=title_final,
    xaxis_title='Предмет',
    yaxis_title='Оценка',
    yaxis=dict(range=[2, 5]),
    barmode='group',
    shapes=[
        # Линия тренда на отметке 4 (темно-серый цвет)
        {
            'type': 'line',
            'x0': -0.5,  # Начало линии по оси X
            'x1': len(subjects) - 0.5,  # Конец линии по оси X
            'y0': 4,
            'y1': 4,
            'line': {
                'color': '#707070',  # Темно-серый цвет
                'width': 2,
                'dash': 'dashdot'  # Пунктирная линия
            }
        }
    ]
)

fig_avg.show()
fig_final.show()

Выгружаем из элетронного журнала оценки за все года обучения

In [124]:
import pandas as pd
import plotly.graph_objects as go

def unify_subject_name(subject):
    subject = str(subject).strip()
    if any(x in subject.lower() for x in ['физкультура', 'физическая культура']):
        return 'Физкультура'
    elif any(x in subject.lower() for x in ['труд', 'технология']):
        return 'Технология'
    elif any(x in subject.lower() for x in ['родной язык', 'родной (русский) язык']):
        return 'Родной (русский) язык'
    elif any(x in subject.lower() for x in ['родная литература', 'литературное чтение на родном']):
        return 'Родная (русская) литература'
    elif any(x in subject.lower() for x in ['литер. чтение', 'литература']):
        return 'Литература'
    elif any(x in subject.lower() for x in ['англ. яз.', 'английский язык', 'иностранный язык (английский)']):
        return 'Иностранный язык (английский)'
    elif any(x in subject.lower() for x in ['информатика', 'введение в информатику']):
        return 'Информатика'
    elif any(x in subject.lower() for x in ['изо', 'изобразительное искусство']):
        return 'ИЗО'
    elif 'всеобщая история' in subject.lower():
        return 'История (всеобщая)'
    else:
        return subject

# Базовый путь к файлам
itog_path = "/content/drive/MyDrive/Оценки/итог/{} класс.xlsx"
classes = range(2, 8)
all_data = []

for class_num in classes:
    file_path = itog_path.format(class_num)
    class_name = f"{class_num} класс"

    try:
        raw_df = pd.read_excel(file_path, header=None)

        header_row = None
        subject_col = None
        year_col = None

        for i in range(min(10, len(raw_df))):
            row = raw_df.iloc[i].astype(str)
            if 'годовая' in row.str.lower().values:
                header_row = i
                year_col = row[row.str.lower().str.contains('годовая')].index[0]
                subject_col = 0
                break

        if header_row is None:
            print(f"Не найдены заголовки в файле {class_name}")
            continue

        df = pd.read_excel(file_path, header=header_row)
        subject_col_name = df.columns[subject_col]
        year_col_name = df.columns[year_col]

        df = df[[subject_col_name, year_col_name]].dropna()
        df[year_col_name] = pd.to_numeric(df[year_col_name], errors='coerce').dropna()

        df[year_col_name] = df[year_col_name].astype(int)
        df[subject_col_name] = df[subject_col_name].apply(unify_subject_name)
        df['Класс'] = class_name
        all_data.append(df.rename(columns={
            subject_col_name: 'Предмет',
            year_col_name: 'Годовая'
        }))

    except Exception as e:
        print(f"Ошибка при обработке {class_name}: {str(e)}")

if not all_data:
    print("Нет данных для обработки")
else:
    combined_df = pd.concat(all_data)

    # Создаем сводную таблицу (классы по строкам, предметы по столбцам)
    pivot_df = combined_df.pivot_table(
        index='Класс',
        columns='Предмет',
        values='Годовая',
        aggfunc='first'
    ).astype('Int64')

    # Упорядочиваем классы
    pivot_df = pivot_df.reindex([f"{i} класс" for i in sorted(classes)])

    # Добавляем средний балл по классам (по строкам)
    pivot_df['Средний по классу'] = pivot_df.mean(axis=1).round(2)

    # Добавляем средний балл по предметам (по столбцам)
    pivot_df.loc['Средний по предмету'] = pivot_df.mean().round(2)

    # Рассчитываем общий средний балл (по всем данным)
    overall_avg = pivot_df.drop('Средний по классу', axis=1)\
                         .drop('Средний по предмету')\
                         .mean().mean().round(2)

    # Заменяем NaN на общий средний балл
    pivot_df.at['Средний по предмету', 'Средний по классу'] = overall_avg

    # Транспонируем для удобного отображения
    display_df = pivot_df.T

    # График (исключаем строки/столбцы со средними)
    plot_df = pivot_df.drop(columns=['Средний по классу']).drop('Средний по предмету')

    fig = go.Figure()
    for class_name in plot_df.index:
        fig.add_trace(go.Bar(
            x=plot_df.columns,
            y=plot_df.loc[class_name],
            name=class_name,
            hoverinfo='y+name'
        ))

    fig.update_layout(
        title='Сравнение оценок по предметам',
        xaxis_title='Предметы',
        yaxis_title='Оценка',
        yaxis=dict(range=[1, 5]),
        height=600,
        barmode='group',
        xaxis=dict(tickangle=45)
    )
    fig.show()

    print("\nСводная таблица с средними баллами:")
    print(f"Общий средний балл: {overall_avg}")
    display(display_df)



Сводная таблица с средними баллами:
Общий средний балл: 4.32


Класс,2 класс,3 класс,4 класс,5 класс,6 класс,7 класс,Средний по предмету
Предмет,,,,,,,
Биология,<NA>,<NA>,<NA>,4.0,4.0,<NA>,4.0
География,<NA>,<NA>,<NA>,4.0,4.0,<NA>,4.0
ИЗО,5.0,5.0,5.0,5.0,5.0,<NA>,5.0
Иностранный язык (английский),5.0,4.0,5.0,4.0,4.0,<NA>,4.4
Иностранный язык (французский язык),<NA>,<NA>,<NA>,3.0,<NA>,<NA>,3.0
Информатика,5.0,4.0,4.0,4.0,<NA>,<NA>,4.25
История (всеобщая),<NA>,<NA>,<NA>,4.0,4.0,<NA>,4.0
История России,<NA>,<NA>,<NA>,<NA>,4.0,<NA>,4.0
Литература,5.0,5.0,5.0,4.0,4.0,<NA>,4.6
